In [ ]:
import logging
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class BasicBot:
    def __init__(self, api_key, api_secret, testnet=True):
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_url = "https://testnet.binancefuture.com" if testnet else "https://fapi.binance.com"
        self.client = Client(api_key, api_secret)
        if testnet:
            self.client.FUTURES_URL = self.base_url

    def place_order(self, symbol, side, order_type, quantity, price=None):
        try:
            order_params = {
                'symbol': symbol,
                'side': SIDE_BUY if side.lower() == 'buy' else SIDE_SELL,
                'type': order_type,
                'quantity': quantity
            }
            if order_type == ORDER_TYPE_LIMIT:
                order_params.update({
                    'price': price,
                    'timeInForce': TIME_IN_FORCE_GTC
                })

            logging.info(f"Placing order: {order_params}")
            order = self.client.futures_create_order(**order_params)
            logging.info(f"Order placed: {order}")
            return order
        except BinanceAPIException as e:
            logging.error(f"Binance API exception: {e}")
        except Exception as e:
            logging.error(f"General exception: {e}")


def get_user_input():
    symbol = input("Enter symbol (e.g., BTCUSDT): ").upper()
    side = input("Enter side (buy/sell): ").lower()
    order_type = input("Enter order type (MARKET/LIMIT): ").upper()
    quantity = float(input("Enter quantity: "))
    price = None
    if order_type == "LIMIT":
        price = float(input("Enter price: "))
    return symbol, side, order_type, quantity, price


def main():
    api_key = os.getenv("BINANCE_API_KEY")
    api_secret = os.getenv("BINANCE_API_SECRET")

    if not api_key or not api_secret:
        api_key = input("Enter Binance API Key: ")
        api_secret = input("Enter Binance API Secret: ")

    bot = BasicBot(api_key, api_secret)
    
    while True:
        try:
            symbol, side, order_type, quantity, price = get_user_input()
            bot.place_order(symbol, side, order_type, quantity, price)
        except KeyboardInterrupt:
            logging.info("Exiting trading bot.")
            break


if __name__ == "__main__":
    main()
